1. This project scrapes the data in a website and stores the results in a text file, `commodity_prices.txt` and a SQLite database,  `CommodityDatabase.db`
2. It stores the current prices of the following commodities :
   1. Crude Oil
   2. Gold
   3. Silver
3. In the SQLite database, the data is stored in a table named `CommodityTable` with the following columns :

| Ticker  | Price  |
| ------- | ------ |
| `TEXT`  | `REAL` |

---

In [1]:
from bs4 import BeautifulSoup
import requests


WEBSITE_URL = "https://finance.yahoo.com/trending-tickers"
data_points_required = ['Crude Oil', 'Gold', 'Silver']
data_points = {}


source = requests.get(WEBSITE_URL).text
soup = BeautifulSoup(source, 'lxml')

for data_point in data_points_required:
  data_point_div = soup.find('li', attrs={'aria-label': data_point})

  if data_point_div is not None:
    data_point_price = data_point_div.h3.find('fin-streamer').text
    data_points[data_point] = data_point_price


---

The following cell creates a `CommodityDatabase.db`, and inserts the data into the table `CommodityTable`.

In [2]:
import sqlite3

DB_NAME = 'CommodityDatabase.db'
TABLE_NAME = 'CommodityTable'

conn = sqlite3.connect(DB_NAME)

c = conn.cursor()

# Drop the CommodityTable table if it exists
c.execute("DROP TABLE IF EXISTS " + TABLE_NAME)

# Create the CommoodityTable table
c.execute("CREATE TABLE " + TABLE_NAME + " (name TEXT, price REAL)")

# Insert data into CommoodityTable
for key, value in data_points.items():
  c.execute("INSERT INTO " + TABLE_NAME + " VALUES (?, ?)", (key, value))

conn.commit()
conn.close()


---

The following cell creates a text file, `commodity_prices.txt` in the directory and stores the data in it.

In [3]:
import os

FILE_NAME = 'commodity_prices.txt'

# Delete the file if it exists
if os.path.exists(FILE_NAME):
  os.remove(FILE_NAME)


with open('commodity_prices.txt', 'w') as f:
  for key, value in data_points.items():
    f.write(key + " : " + value + "\n")
